# Memory in Agent
---
En este cuaderno vamos a agregar memoria a un Agente


Para agregar una memoria a un agente vamos a seguir los siguientes pasos:

In [1]:
from dotenv import load_dotenv, find_dotenv
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.utilities import SerpAPIWrapper, WikipediaAPIWrapper

load_dotenv(find_dotenv())

True

In [13]:
search = SerpAPIWrapper()
wikipedia = WikipediaAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need answer questions about current events",
    ),
    # Tool(
    #     name="Wikipedia",
    #     func=wikipedia.run,
    #     description="useful for when you need answer questions using wikipedia",
    # )
]

In [4]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
)

Ahora podemos construir LLMChain, con el objeto memoria, y luego crear el agente.

In [6]:
# llm 
llm = OpenAI(temperature=0)

# llm chain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
)

# agent
agent = ZeroShotAgent(
    llm_chain=llm_chain,
    tools=tools,
    verbose=True,
)

# agent executor
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
)

In [7]:
agent_chain.run(input="How many people live in canada?")



> Entering new AgentExecutor chain...
Thought: I should look up the population of Canada
Action: Search
Action Input: Population of Canada
Observation: {'type': 'population_result', 'place': 'Canada', 'population': '38.25 million', 'year': '2021'}
Thought: I now know the final answer
Final Answer: Canada has a population of 38.25 million people in 2021.

> Finished chain.


'Canada has a population of 38.25 million people in 2021.'

Para probar la memoria de este agente, podemos hacer una pregunta de seguimiento que se base en la información del intercambio anterior para ser respondida correctamente.

In [8]:
agent_chain.run(input="what is their national anthem called?")



> Entering new AgentExecutor chain...
Thought: I need to find out what the national anthem of Canada is called.
Action: Search
Action Input: "national anthem of Canada"
Observation: ['"O Canada" is the national anthem of Canada. The song was originally commissioned by Lieutenant Governor of Quebec Théodore Robitaille for the 1880 Saint-Jean-Baptiste Day ceremony; Calixa Lavallée composed the music, after which words were written by the poet and judge Sir Adolphe-Basile Routhier.', 'O Canada type: National anthem.', 'O Canada main_tab_text: Overview.', 'O Canada kgmid: /m/05mtn.', 'O Canada lyricists: Calixa Lavallée, Robert Stanley Weir, Adolphe-Basile Routhier.', 'O Canada composer: Calixa Lavallée.', 'O Canada key: G major.', 'O Canada adopted: July 1, 1980.', 'O Canada also_known_as: French - Ô Canada.', 'It was proclaimed the official national anthem on July 1, 1980. “God Save the Queen” remains the royal anthem of Canada. The music, written by ...']
Thought: I now know the final

'The national anthem of Canada is called "O Canada".'

Podemos ver que el agente recordó que la pregunta anterior era sobre Canadá y preguntó correctamente a la Búsqueda de Google cuál era el nombre del himno nacional de Canadá.

Para divertirnos, comparemos esto con un agente que NO tiene memoria.

In [14]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, prefix=prefix, suffix=suffix, input_variables=["input", "agent_scratchpad"]
)

llm_chain = LLMChain(
    llm=OpenAI(temperature=0), 
    prompt=prompt
)

agent = ZeroShotAgent(
    llm_chain=llm_chain, 
    tools=tools, 
    verbose=True
)

agent_without_memory = AgentExecutor.from_agent_and_tools(
    agent=agent, 
    tools=tools, 
    verbose=True
)

In [15]:
agent_without_memory.run("How many people live in canada?")



> Entering new AgentExecutor chain...
Thought: I should look up the population of Canada
Action: Search
Action Input: Population of Canada
Observation: {'type': 'population_result', 'place': 'Canada', 'population': '38.25 million', 'year': '2021'}
Thought: I now know the final answer
Final Answer: Canada has a population of 38.25 million people in 2021.

> Finished chain.


'Canada has a population of 38.25 million people in 2021.'

In [16]:
agent_without_memory.run("what is their national anthem called?")



> Entering new AgentExecutor chain...
Thought: I should look up the national anthem of the country
Action: Search
Action Input: National anthem of [country]
Observation: ['Most nation states have an anthem, defined as "a song, as of praise, devotion, or patriotism"; most anthems are either marches or hymns in style.']
Thought: I should look for the specific name of the anthem
Action: Search
Action Input: Name of national anthem of [country]
Observation: ['^ Jump up to: "God Save the King" is the royal anthem of Antigua and Barbuda, Australia, the Bahamas, Canada, Jamaica, Papua New Guinea, Saint Kitts and Nevis, ...']
Thought: I now know the final answer
Final Answer: The national anthem of [country] is called "God Save the King".

> Finished chain.


'The national anthem of [country] is called "God Save the King".'